In [1]:
!pip install tensorflow-addons
!pip install --upgrade tensorflow
!pip install --upgrade scikit-learn
!pip install -U keras-tuner


Requirement already up-to-date: tensorflow in /usr/local/lib/python3.6/dist-packages (2.2.0rc2)
Requirement already up-to-date: scikit-learn in /usr/local/lib/python3.6/dist-packages (0.22.2.post1)
     |████████████████████████████████| 61kB 2.0MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=1727621ed3c22913dd2261c36bc6f2ec4b3cd2ee3817ca001b76a12f7f424517
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=958f65607cc8a72db108489b4bc042916450118de03c4f1a4a55e589a76d2c3a
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [6]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_addons as tfa
from tensorflow.keras import Model
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input,Dense,Dropout,Embedding,LSTM,Bidirectional,Flatten,Conv1D,Reshape, TimeDistributed,Reshape,Activation,Dot,MaxPool1D,GlobalMaxPool1D,GRU,concatenate,GlobalAveragePooling1D
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import io
from google.colab import files
from google.colab import drive
from numpy import array
from numpy import asarray
from numpy import zeros
from numpy import random as random
import keras
import keras.backend as K
import gensim
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

Using TensorFlow backend.


In [7]:
from google.colab import drive
drive.mount('/content/drive')
uploaded = files.upload()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
f = '/content/drive/My Drive/sadTextv3 - sadTextv3.csv'
df = pd.read_csv(f, sep=',', error_bad_lines=False)
df = df[df['text'].notnull()]

In [0]:
df["text"] = df["text"].str.lower()
input_sentences = [text.split(" ") for text in df["text"].values.tolist()]
labels = df["emotion"].values.tolist()

In [16]:
word2id=dict()
label2id=dict()
max_words = 0

for sentence in input_sentences:
  for word in sentence:
    word = word.replace(".","")
    if word not in word2id:
      word2id[word] = len(word2id)

    if len(sentence) > max_words:
      max_words = len(sentence)

label2id = {l: i for i, l in enumerate(set(labels))}
id2label = {v:k for k, v in label2id.items()}
id2label


{0: 'sadness',
 1: 'shame',
 2: 'suffering',
 3: 'neglect',
 4: 'sympathy',
 5: 'disappointment'}

In [17]:
#Encoding Sentences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
Y = [label2id[label] for label in labels]
t = Tokenizer()
t.fit_on_texts(input_sentences)
vocab_size = len(t.word_index)+1
print(vocab_size)

encoded_data = t.texts_to_sequences(input_sentences)
padded_data = pad_sequences(encoded_data, max_words)
Y = to_categorical(Y, num_classes=len(label2id), dtype='float32')
print("Shape of X: {}".format(padded_data.shape))
print("Shape of Y: {}".format(Y.shape))
print(t.word_index)

2008
Shape of X: (878, 57)
Shape of Y: (878, 6)
{'i': 1, 'to': 2, 'and': 3, 'my': 4, 'the': 5, 'me': 6, 'a': 7, 'of': 8, 'that': 9, 'it': 10, 'but': 11, 'just': 12, 'in': 13, 'for': 14, 'is': 15, 'have': 16, 'feel': 17, 'this': 18, 'like': 19, 'so': 20, 'you': 21, 'with': 22, 'was': 23, "i'm": 24, 'not': 25, 'be': 26, 'want': 27, 'all': 28, 'im': 29, 'no': 30, 'life': 31, 'do': 32, 'about': 33, 'even': 34, 'myself': 35, 'how': 36, 'because': 37, 'or': 38, 'dont': 39, 'time': 40, 'now': 41, 'at': 42, "don't": 43, 'on': 44, 'never': 45, 'can': 46, 'he': 47, 'when': 48, 'they': 49, 'know': 50, 'really': 51, 'am': 52, 'as': 53, 'people': 54, 'being': 55, 'her': 56, 'if': 57, 'would': 58, 'out': 59, 'one': 60, 'get': 61, 'had': 62, 'from': 63, "can't": 64, 'we': 65, 'been': 66, 'depression': 67, 'up': 68, 'what': 69, 'its': 70, 'she': 71, 'much': 72, 'only': 73, 'are': 74, 'anything': 75, 'could': 76, 'them': 77, 'nothing': 78, 'alone': 79, 'cant': 80, 'has': 81, 'help': 82, 'him': 83, 'mor

In [12]:
from google.colab import drive
from numpy import array
from numpy import asarray
from numpy import zeros
#Loading embedding file into memory
print('Indexing word vectors.')

embeddings_index = {}
f = open('/content/drive/My Drive/glove.6B/glove.6B.300d.txt')
for line in f:
    values = line.split()
    word = values[0]
    vec = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = vec
f.close()

print('Found %s word vectors.' % len(embeddings_index))


Indexing word vectors.
Found 400000 word vectors.


In [0]:
embedding_matrix  = zeros((vocab_size,300))
for word, i in t.word_index.items():
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector


Building a model for use with Keras Tuner,
Restart before using model.fit as it appears to interfere with it


In [0]:
'''import kerastuner as kt

def build_tuner_model(hp):

  embedding_dim = 300
  
  inputs=Input(shape=(max_words,))
  x = inputs
  x = Embedding(vocab_size,embedding_dim, weights=[embedding_matrix],input_length=max_words,trainable=False)(x)
  
  x = Bidirectional((GRU(hp.Int('lstmunits',100,300,step=50),activation='tanh',return_sequences = True)))(x)

  a = Dense(1,activation='tanh')(x)
  a = Activation('softmax', name='attention_vec')(x)
  x = Dot(axes=1)([x, a])

  x = Bidirectional((GRU(hp.Int('lstmunits',50,150,step=25),activation='tanh',return_sequences = False)))(x)

  x = Dropout(hp.Float('dropout',0.1,0.5,step=0.1,default=0.5))(x)

  x = Dense(hp.Int('fc units',100,200,step=10, default=100), activation='relu')(x)
  x = Dropout(hp.Float('dropout2', 0.1, 0.5, step=0.1, default=0.5))(x)
  outputs = Dense(len(label2id), activation = 'softmax')(x)

  model = Model(inputs = [inputs], outputs=outputs)
  model.compile(optimizer=tf.keras.optimizers.Adam(
                hp.Float('learning_rate', 1e-4, 1e-2, sampling='log')),
                loss='categorical_crossentropy',
                metrics=['accuracy'])
  return model'''

Model function 

In [0]:

def createmodel():
  embedding_dim = 300

  inputs=Input(shape=(max_words,),dtype='int32')


  xx = Embedding(vocab_size,embedding_dim, weights=[embedding_matrix],input_length=max_words,trainable=False)(inputs)
  xx = Bidirectional((GRU(300,activation='tanh',return_sequences = True)))(xx)

  av = Dense(1,activation='tanh')(xx)
  av = Activation('softmax', name='attention_vec')(av)
  aa = Dot(axes=1)([xx, av])
  
  xx = Bidirectional((GRU(100,activation='tanh',return_sequences=False)))(aa)

  dr = Dropout(0.3)(xx)

  fc = Dense(160, activation='relu')(dr)
  dr = Dropout(0.1)(fc)
  out = Dense(len(label2id), activation = 'softmax')(dr)

  model = Model(inputs = [inputs], outputs = out)
  model.compile(optimizer=tf.keras.optimizers.Adam(0.004370187549970498),
                loss='categorical_crossentropy',
                metrics=['accuracy',tf.keras.metrics.Recall(),
                        tf.keras.metrics.Precision(),])
  model.summary()

  return model

In [0]:
#Callbacks
def scheduler(epoch):
  if epoch < 20:
    return 0.001
  else:
    return 0.001 * tf.math.exp(0.1 * (10 - epoch))

best_epoch = 'weights.{epoch:02d}-{val_accuracy:.2f}.hdf5'
checkpoint =  tf.keras.callbacks.ModelCheckpoint(filepath=best_epoch, monitor='val_accuracy',save_best_only=True,mode='max')
scheduler_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10, mode="max")
callbacks = [earlystop,checkpoint]


In [0]:
test_split = 650
X_train=padded_data[:test_split]
Y_train=Y[:test_split]
X_test=padded_data[test_split+1:]
Y_test=Y[test_split+1:] 

Hyperband Tuning.Skip this if parameters are already tuned

In [0]:
'''tuner = kt.Hyperband(
    build_tuner_model,
    objective='val_accuracy',
    max_epochs=30,
    hyperband_iterations=10,
    project_name='test2')
'''

In [0]:
'''tuner.search(X_train,Y_train,
             validation_split=0.5,
             shuffle = True,
             epochs=30,
             callbacks=[earlystop])
'''

In [0]:
'''best_hyperparameters = tuner.get_best_hyperparameters(1)[0]
print(best_hyperparameters)
'''

Real model training.

In [0]:
model = createmodel()
history = model.fit(X_train,Y_train, epochs=100, batch_size=128, validation_split=0.2, shuffle=True, callbacks=callbacks)


In [0]:
#converting Y_test from one hot encoding for sklearn classification_report
Y_onehot = Y_test.tolist()
Y_true=[]

for items in Y_onehot:
  i=0
  for item in items: 
    if item == 1:
      Y_true.append(i)
      
    else:
      i+=1

In [0]:
#Loading the weights from the best epoch
#model.load_weights('/content/weights.09-0.37.hdf5')

preds=model.predict(X_test, batch_size=32, verbose=1)
pred_classes = np.argmax(preds,axis=1)
labels = ['sadness','disappointment','suffering','shame','neglect','sympathy']
print(classification_report(Y_true,pred_classes,labels=[0,1,2,3,4,5,6],target_names=labels,digits=4))

In [0]:
tf.keras.utils.plot_model(createmodel(), show_shapes=True, show_layer_names=True, to_file='model.png')

In [38]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Bi-LSTM Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Bi-LSTM loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()



NameError: ignored

In [39]:
test_split = 650

labels = df["emotion"].values.tolist()
newpreds = [id2label[item] for item in pred_classes]
print(len(newpreds))
print(len(labels[test_split+1:]))
text=df["text"][test_split+1:].tolist()
print(len(text))
data = {'Text':text, 'Classification':labels[test_split+1:], 'Prediction': newpreds} 
newdf = pd.DataFrame(data)
newdf.to_csv("predscnn.csv")

227
227
227
